# Load Pickle

In [ ]:
import os
import sys
sys.path.append("../..")

from typing import Tuple
import pandas as pd

from utils.io import load_yaml

server = "local"
server = load_yaml("../../config/servers.yml")[server]

database = server["database"]["location"]

def pickle_path(name: str) -> Tuple[str]:
    prefix = ""
    motion_pickle = ""
    if name == "cholec80":
        prefix = "cholec80_single_video_frames_cropped"
        motion_pickle = "23_02_24_pre_processed_frame_increment_5_frames_between_clips_1_log.pkl"
    if name == "heichole":
        prefix = "heichole_single_frames_cropped"
        motion_pickle = "23_02_24_pre_processed_frame_increment_5_frames_between_clips_1_log.pkl"
    if name == "phantom":
        prefix = "21_11_25_first_test_data_frames"
        motion_pickle = "23_02_24_pre_processed_frame_increment_10_frames_between_clips_1_log.pkl"
    if name == "autolaparo":
        prefix = "autolaparo_single_frames/AutoLaparo_Task2"
        motion_pickle = "23_02_24_pre_processed_frame_increment_5_frames_between_clips_1_log.pkl"

    return prefix, motion_pickle

# cholec80
# prefix, motion_pickle = pickle_path("cholec80")
# prefix, motion_pickle = pickle_path("heichole")
prefix, motion_pickle = pickle_path("phantom")
# prefix, motion_pickle = pickle_path("autolaparo")


df = pd.read_pickle(os.path.join(database, prefix, motion_pickle))
df = df.dropna()

In [ ]:
from utils.processing import dataframe_duv_running_average, classify_duv_motion

window = 40 # window 20 for 25 fps videos
df_ra, mean, std = dataframe_duv_running_average(df, window=window)
print(mean, std)
df_ra

In [ ]:
df_ra = df_ra.dropna()
# df_ra["labels"] = df_ra.apply(lambda x: classify_duv_motion(x["duv_0_0"], x["duv_0_1"], x["duv_1_0"], x["duv_1_1"], x["duv_2_0"], x["duv_2_1"], x["duv_3_0"], x["duv_3_1"], mean + std), axis=1) # window of 1
df_ra["labels"] = df_ra.apply(lambda x: classify_duv_motion(x["duv_0_0"], x["duv_0_1"], x["duv_1_0"], x["duv_1_1"], x["duv_2_0"], x["duv_2_1"], x["duv_3_0"], x["duv_3_1"], mean + std/5.), axis=1) # window of 0.8s
df_ra

In [ ]:
# remove duv columns
df_ra = df_ra.drop(columns=[f"duv_{i}_{j}" for i in range(4) for j in range(2)])

# rename and save
string_split = motion_pickle.split("_")
date, new_name = "_".join(string_split[:3]), "_".join(string_split[3:])
new_name = new_name.replace("pre_processed_", "")
new_name = date + f"_motion_label_window_{window}_" + new_name

df_ra.to_pickle(os.path.join(database, prefix, new_name))

In [ ]:
df_ra.groupby("labels").size()

In [ ]:
# split into test/train
import sys
sys.path.append("../..")

import os
from typing import Tuple
import pandas as pd

from utils.io import load_yaml
from utils.processing import unique_video_train_test

window = 40 # window 20 for 25 fps videos

def pickle_path(name: str) -> Tuple[str]:
    prefix = ""
    motion_pickle = ""
    if name == "cholec80":
        prefix = "cholec80_single_video_frames_cropped"
        motion_pickle = f"23_02_24_motion_label_window_{window}_frame_increment_5_frames_between_clips_1_log.pkl"
    if name == "heichole":
        prefix = "heichole_single_frames_cropped"
        motion_pickle = f"23_02_24_motion_label_window_{window}_frame_increment_5_frames_between_clips_1_log.pkl"
    if name == "phantom":
        prefix = "21_11_25_first_test_data_frames"
        motion_pickle = f"23_02_24_motion_label_window_{window}_frame_increment_10_frames_between_clips_1_log.pkl"
    if name == "autolaparo":
        prefix = "autolaparo_single_frames/AutoLaparo_Task2"
        motion_pickle = f"23_02_24_motion_label_window_{window}_frame_increment_5_frames_between_clips_1_log.pkl"

    return prefix, motion_pickle

server = "local"
server = load_yaml("../../config/servers.yml")[server]

database = server["database"]["location"]

# datasets = ["cholec80", "heichole", "autolaparo"]
datasets = ["phantom"]

for dataset in datasets:
    prefix, motion_pickle = pickle_path(dataset)

    df = pd.read_pickle(os.path.join(database, prefix, motion_pickle))

    # remove train volumn
    df = unique_video_train_test(df, train_split=0.8, tolerance=0.01, random_state=42)

    print(dataset, ": ", len(df[df["train"] == False])/len(df))

    motion_pickle_test_train = motion_pickle.split(".")[0] + "_test_train.pkl"
    df.to_pickle(os.path.join(database, prefix, motion_pickle_test_train))